In [ ]:
!pip install kaggle

In [ ]:
!pip install rouge

In [13]:
import pandas as pd
import numpy as np
from rouge import Rouge

In [ ]:
rouge = Rouge()

In [ ]:
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail

Dataset URL: https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail
License(s): CC0-1.0
 99% 500M/503M [00:16<00:00, 39.2MB/s]
100% 503M/503M [00:16<00:00, 31.6MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/newspaper-text-summarization-cnn-dailymail.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [7]:
df_train=pd.read_csv('/content/cnn_dailymail/train.csv')
df_test=pd.read_csv('/content/cnn_dailymail/test.csv')
df_val=pd.read_csv('/content/cnn_dailymail/validation.csv')

In [8]:
sample_text = df_train['article'][100]
sample_summary = df_train['highlights'][100]
sample_text

"By . John Drayton . Lionel Messi took matters into his own hands and delivered Argentina's team talk following a goalless 90 minutes against Holland. Manager Alejandro Sabella began the chat but Messi soon took over as his team-mates listened intently. And although his side couldn’t muster a win in extra time, they edged out Holland 4-2 on penalties to reach Sunday’s final in Rio de Janeiro. VIDEO Scroll down to watch Mascherano hailed the hero as Buenos Aires celebrates . Follow the leader: Lionel Messi (centre) led Argentina's team talk before extra time . Stepping up to the plate: Javier Mascherano led the team talk at half-time during extra time . Elation: Messi is overjoyed after Argentina's shoot-out win over Holland in the World Cup semi-final . Leading the celebrations: Messi (left) runs along with his team-mates to celebrate their win . As cool as you like: Messi slots home his penalty during the shoot-out as Argentina win . Leading from the front: Messi (second left) takes o

In [9]:
sample_summary

"Messi led the Argentina team talk between full-time and extra-time .\nJavier Mascherano took over for half-time of extra-time .\nHe also led the team talk before the side stepped up for penalties .\nArgentina won 4-2 on penalties and will play Germany in Sunday's final .\nIt will be the third time that Germany and Argentina face each other in a World Cup final ."

In [10]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load BART model and tokenizer
model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

def summarize_bart(text):
    inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt")
    summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=200, min_length=50, length_penalty=2.0, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [14]:
bart_summary = summarize_bart(sample_text)
bart_summary

"Argentina beat Holland 4-2 on penalties in World Cup semi-final in Sao Paulo. Lionel Messi led Argentina's team talk between normal time and extra time. Javier Mascherano led the team talk at half-time. Messi had a quiet game but scored four goals in the tournament. Argentina face Germany in Sunday's final in Rio de Janeiro."

In [15]:
scores = rouge.get_scores([sample_summary], [bart_summary], avg=True)

# Print the ROUGE scores
print(f"ROUGE-1: {scores['rouge-1']['f']:.4f}")
print(f"ROUGE-2: {scores['rouge-2']['f']:.4f}")
print(f"ROUGE-L: {scores['rouge-l']['f']:.4f}")

ROUGE-1: 0.5227
ROUGE-2: 0.2342
ROUGE-L: 0.5000


In [18]:
summaries_of_bart = []
references_for_bart= []

for article in df_train['article'][:1000]:
    summary = summarize_bart(article)
    summaries_of_bart.append(summary)

for highlight in df_train['highlights'][:1000]:
    references_for_bart.append(highlight)

scores = rouge.get_scores(summaries_of_bart, references_for_bart, avg=True)

print(f"ROUGE-1: {scores['rouge-1']['f']:.4f}")
print(f"ROUGE-2: {scores['rouge-2']['f']:.4f}")
print(f"ROUGE-L: {scores['rouge-l']['f']:.4f}")

ROUGE-1: 0.3867
ROUGE-2: 0.1805
ROUGE-L: 0.3645
